<h1 align="center"> Titanic: Machine Learning from Disaster </h1>

<h3 align="center"> Logistic Regression </h3>

<h3 align="center"> W Cui </h3>

In [1]:
# Import modules
import pandas as pd
import statsmodels.api as sm
from patsy import dmatrices
import matplotlib.pyplot as plt

# Configure inline mode
%matplotlib inline

We use the titanic dataset to show how to conduct logistic regression in Python.

In [2]:
dat = pd.read_csv("../Data/titanic.csv")
dat.head()

,pclass,survived,sex,age,sibsp,parch,fare
0,1,1,female,29.0000,0,0,211.3375
1,1,1,male,0.9167,1,2,151.5500
2,1,0,female,2.0000,1,2,151.5500
3,1,0,male,30.0000,1,2,151.5500
4,1,0,female,25.0000,1,2,151.5500


In [3]:
# Summary statistics
dat.describe()

,pclass,survived,age,sibsp,parch,fare
count,1309.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000
mean,2.294882,0.381971,29.881135,0.498854,0.385027,33.295479
std,0.837836,0.486055,14.413500,1.041658,0.865560,51.758668
min,1.000000,0.000000,0.166700,0.000000,0.000000,0.000000
25%,2.000000,0.000000,21.000000,0.000000,0.000000,7.895800
50%,3.000000,0.000000,28.000000,0.000000,0.000000,14.454200
75%,3.000000,1.000000,39.000000,1.000000,0.000000,31.275000
max,3.000000,1.000000,80.000000,8.000000,9.000000,512.329200


In [4]:
# Correlation matrix
dat.corr()

,pclass,survived,age,sibsp,parch,fare
pclass,1.000000,-0.312469,-0.408106,0.060832,0.018322,-0.558629
survived,-0.312469,1.000000,-0.055513,-0.027825,0.082660,0.244265
age,-0.408106,-0.055513,1.000000,-0.243699,-0.150917,0.178739
sibsp,0.060832,-0.027825,-0.243699,1.000000,0.373587,0.160238
parch,0.018322,0.082660,-0.150917,0.373587,1.000000,0.221539
fare,-0.558629,0.244265,0.178739,0.160238,0.221539,1.000000


## 1. Frequency Table

Create frequency tables of the variable "survived".

In [5]:
# Frequency table
pd.crosstab(dat.survived, columns='count')

col_0,count
survived,
0,809
1,500


In [6]:
# Relative frequency table
pd.crosstab(dat.survived, columns='count')/pd.crosstab(dat.survived, columns='count').sum()

col_0,count
survived,
0,0.618029
1,0.381971


## 2. Create Design Matrices

To fit most of the models covered by statsmodels, you will need to create two design matrices. The first is a matrix of endogenous variable(s) (i.e. dependent, response, regressand, etc.). The second is a matrix of exogenous variable(s) (i.e. independent, predictor, regressor, etc.). 

The patsy module provides a convenient function to prepare design matrices using R-like formulas. You can find more information here: http://patsy.readthedocs.org

We use patsy's dmatrices function to create design matrices:

In [7]:
# Create design matrices
y, X = dmatrices('survived ~ pclass + sex + age + sibsp + parch + fare',
                 data=dat,
                 return_type='dataframe')

In [8]:
# show head of y
y.head()

,survived
0,1.0
1,1.0
2,0.0
3,0.0
4,0.0


In [9]:
# show head of X
X.head()

,Intercept,sex[T.male],pclass,age,sibsp,parch,fare
0,1.0,0.0,1.0,29.0000,0.0,0.0,211.3375
1,1.0,1.0,1.0,0.9167,1.0,2.0,151.5500
2,1.0,0.0,1.0,2.0000,1.0,2.0,151.5500
3,1.0,1.0,1.0,30.0000,1.0,2.0,151.5500
4,1.0,0.0,1.0,25.0000,1.0,2.0,151.5500


## 3. Fit Model with Data

Fitting a model in statsmodels typically involves 3 easy steps:

- Use the model class to describe the model
- Fit the model using a class method
- Inspect the results using a summary method

In [10]:
# Describe the logistic model
mod = sm.Logit(y,X)

# Fit model
fit = mod.fit()

# Summarize model
fit.summary()

Optimization terminated successfully.
         Current function value: 0.464293
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               survived   No. Observations:                 1045
Model:                          Logit   Df Residuals:                     1038
Method:                           MLE   Df Model:                            6
Date:                Mon, 27 Feb 2017   Pseudo R-squ.:                  0.3135
Time:                        04:44:29   Log-Likelihood:                -485.19
converged:                       True   LL-Null:                       -706.79
                                        LLR p-value:                 1.428e-92
===============================================================================
                  coef    std err          z      P>|z|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------
Intercept       4.7764      0.478      9.992      0.000         3.839     5.713
sex[T.male]    -2.5467      0.173    -14.682      0.000        -2.887    -2.207
pclass         -1.0982      0.131     -8.364      0.000        -1.356    -0.841
age            -0.0384      0.007     -5.862      0.000        -0.051    -0.026
sibsp          -0.3547      0.106     -3.351      0.001        -0.562    -0.147
parch           0.0549      0.103      0.533      0.594        -0.147     0.257
fare            0.0016      0.002      0.865      0.387        -0.002     0.005
===============================================================================
"""

In [11]:
# Show coefficient estimates
fit.params

Intercept      4.776374
sex[T.male]   -2.546715
pclass        -1.098227
age           -0.038412
sibsp         -0.354726
parch          0.054925
fare           0.001641
dtype: float64

In [12]:
# Show in-sample prediction table (confusion matrix)
fit.pred_table()

array([[ 522.,   96.],
       [ 127.,  300.]])

The coefficients of the discrete choice model do not tell us much. Here, we show average marginal effects.

In [13]:
# To diable warnings
import warnings
warnings.filterwarnings("ignore")

# Calculate average marginal effects
mfx = fit.get_margeff()
print(mfx.summary())

        Logit Marginal Effects       
Dep. Variable:               survived
Method:                          dydx
At:                           overall
                 dy/dx    std err          z      P>|z|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------
sex[T.male]    -0.3807      0.014    -27.057      0.000        -0.408    -0.353
pclass         -0.1642      0.018     -9.279      0.000        -0.199    -0.129
age            -0.0057      0.001     -6.174      0.000        -0.008    -0.004
sibsp          -0.0530      0.016     -3.407      0.001        -0.084    -0.023
parch           0.0082      0.015      0.533      0.594        -0.022     0.038
fare            0.0002      0.000      0.866      0.386        -0.000     0.001


We find that the average marginal effect of being male on survival is negative: -0.381. This means that the probability of survival is on average about 38 percentage points lower for male than for female, holding all other variables constant.

## 4. Predict New Data

We collect test data for Jack and Rose from the plot of the movie https://en.wikipedia.org/wiki/Titanic_(1997_film)


In [14]:
X.head()

,Intercept,sex[T.male],pclass,age,sibsp,parch,fare
0,1.0,0.0,1.0,29.0000,0.0,0.0,211.3375
1,1.0,1.0,1.0,0.9167,1.0,2.0,151.5500
2,1.0,0.0,1.0,2.0000,1.0,2.0,151.5500
3,1.0,1.0,1.0,30.0000,1.0,2.0,151.5500
4,1.0,0.0,1.0,25.0000,1.0,2.0,151.5500


In [15]:
# Create a new dataset for Jack and Rose
New_X = pd.DataFrame([[1,1,3,19,0,0,5],
                      [1,0,1,17,0,1,500]])
New_X.columns = ["Intercept","sex[T.male]","pclass","age","sibsp","parch","fare"]
New_X.index = ['Jack','Rose']

New_X

,Intercept,sex[T.male],pclass,age,sibsp,parch,fare
Jack,1,1,3,19,0,0,5
Rose,1,0,1,17,0,1,500


In [16]:
# Predict survival probability of Jack and Rose
New_X['Pred'] = fit.predict(exog=New_X)

New_X

,Intercept,sex[T.male],pclass,age,sibsp,parch,fare,Pred
Jack,1,1,3,19,0,0,5,0.143484
Rose,1,0,1,17,0,1,500,0.980167
